## Задание 1.

Обязательная часть\
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)\
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [49]:
def habr_parse(df, keywords, pages=1):      

    #Цикл сколько страниц обработать     
    for page in range(pages):
        
        print('Обрабатываю страницу номер ' + str(page+1))
        
        if page == 0:
            res = requests.get('https://habr.com/ru/all/')
            soup = BeautifulSoup(res.text, 'html.parser')
            posts = soup.find_all('article', class_='post post_preview')
        else:
            next_page_link = 'https://habr.com' + soup.find('a', class_='arrows-pagination__item-link arrows-pagination__item-link_next').attrs.get('href')
            res = requests.get(next_page_link)
            soup = BeautifulSoup(res.text, 'html.parser')
            posts = soup.find_all('article', class_='post post_preview')
            
        for Post in posts:

            #Дата поста     
            Post_date = Post.find('span', class_='post__time').get_text()

            #Заголовок поста 
            Post_title_txt = Post.find('a', class_='post__title_link').get_text()

            #Ссылка на пост     
            Post_title_link = Post.find('a', class_='post__title_link').attrs.get('href')

            #Открываю сам пост
            GetPost = requests.get(Post_title_link)
            Post_soup = BeautifulSoup(GetPost.text, 'html.parser')

            try:
                Post_full_txt = Post_soup.find('div', 'post__text post__text-html post__text_v1').get_text()
                print('Получилось!')
            except:
                print('Ищу альтернативу')
                Post_full_txt = Post_soup.find('div', 'post__text post__text_v2').get_text()
                print('Получилось!')

            #Ищу ключевые слова в заголовке или тексте
            if any([key in Post_title_txt.lower() for key in KEYWORDS]):
                row = {'Post_date': Post_date, 'Post_title_txt': Post_title_txt, 'Post_title_link': Post_title_link, 'Post_full_txt': Post_full_txt}
                df = pd.concat([df, pd.DataFrame([row])]) 
            elif any([key in Post_full_txt.lower() for key in KEYWORDS]):
                row = {'Post_date': Post_date, 'Post_title_txt': Post_title_txt, 'Post_title_link': Post_title_link, 'Post_full_txt': Post_full_txt}
                df = pd.concat([df, pd.DataFrame([row])])
                
        
            
    return df

In [50]:
#Что ищем
KEYWORDS = ['python', 'парсинг']
#Количество страниц
Pages = 5
#Пустой дата фрейм
habr = pd.DataFrame({})

#Поехали
habr =  habr_parse(habr, KEYWORDS, Pages)
habr.head()

Обрабатываю страницу номер 1
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Получилось!
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Обрабатываю страницу номер 2
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
Получилось!
Получилось!
Обрабатываю страницу номер 3
Ищу альтернативу
Получилось!
Ищу альтернативу
Получилось!
И

,Post_date,Post_title_txt,Post_title_link,Post_full_txt
0,сегодня в 07:51,HackTheBox. Прохождение Feline. RCE через сери...,https://habr.com/ru/post/542612/,Продолжаю публикацию решений отправленных на д...
0,вчера в 18:15,Поддержка токенов PKCS#11 с ГОСТ-криптографией...,https://habr.com/ru/post/542182/,Поддержка криптографических токенов PKCS#11 с ...
0,вчера в 15:43,Почему алгоритмы ML так трудно настраивать?,https://habr.com/ru/company/skillfactory/blog/...,\n\r\nВ машинном обучении линейные комбинации ...
0,вчера в 14:28,«Как QA в управлении хранилища данных эволюцио...,https://habr.com/ru/company/tinkoff/blog/543416/,Часть 1. Прошлое. Переломные моментыПрофессион...
0,вчера в 12:41,Автоматизация рутины в DotA 2 или читы?,https://habr.com/ru/post/543354/,"Привет, в этой статье будут рассматриваться ле..."


## Задание 2.
## Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

## Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [103]:
import json

mails = ['sashasaxanova@gmail.com', 'chelnokovpavel@gmail.com', 'pashasaxanov@mail.ru']
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
param = {'emailAddresses': mails}
header = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}
breaches_df = pd.DataFrame({})
summary_df = pd.DataFrame({})

#Обращаюсь к сайту 1 раз и запрашиваю список, чтобы не передавать по 1 мейлу в цилке
#Пару раз меня залочило, мне хватило
# avast_res = requests.post(URL, data = json.dumps(param), headers = header)
# print(avast_res)

# Читаю json, формирую df
for main_key, main_val in avast_res.json().items(): 
    
    #Собираю df формата breachId, site, description, publishDate
    if main_key == 'breaches':
        
        for breaches_key, breaches_val in main_val.items():
            
            breaches_row = {'breachId': breaches_val['breachId'],
                            'site': breaches_val['site'],\
                            'description': breaches_val['description'],\
                            'publishDate': breaches_val['publishDate']}
            breaches_df = pd.concat([breaches_df, pd.DataFrame([breaches_row])])
            
    #Собираю df формата breachId, mail. Чтобы потом объединить дата фреймы            
    if main_key == 'summary':
        
        for summary_key, summary_val in main_val.items():
#             print(summary_key)
#             print(summary_val)

            for summary_val2 in summary_val.values():
        
                for summary_val3 in summary_val2:
#                     print(summary_val2)
#                     print(summary_key)

                    summary_row = {'breachId': summary_val3,\
                                   'mail': summary_key}
                    summary_df = pd.concat([summary_df, pd.DataFrame([summary_row])])   
        
summary_df.set_index('breachId').join(breaches_df.set_index('breachId'), how='left')

,mail,site,description,publishDate
breachId,,,,
16665,sashasaxanova@gmail.com,shein.com,"In June 2018, fast-fashion retailer Shein's da...",2019-05-16T00:00:00Z
12,chelnokovpavel@gmail.com,vk.com,Popular Russian social networking platform VKo...,2016-10-29T00:00:00Z
13535,chelnokovpavel@gmail.com,dayz.com,"In April 2016, DayZ Forum's database was alleg...",2018-02-03T00:00:00Z
13916,chelnokovpavel@gmail.com,myfitnesspal.com,"In February 2018, MyFitnessPal's database was ...",2018-03-20T00:00:00Z
17822,chelnokovpavel@gmail.com,synergyregions.ru,"At an unconfirmed date, the Russian university...",2020-07-16T00:00:00Z
